# Using a TensorNode class to insert a network built with the Tensorflow Keras API into a Nengo model
This tutorial builds on [Inserting a TensorFlow network into a Nengo model](./pretrained-model.ipynb). In that tutorial, we showed how to write a TensorNode class that makes it easy to insert a pre-trained network from `tf.contrib.slim` into a Nengo model.  
Sometimes, instead of using a pre-trained network, you may want to use your own neural net architecture within a TensorNode. Here we show how a network built with the `tf.keras` API can be insterted into a Nengo model using the TensorNode class. If you haven't read the previous tutorial, please work through that one to familiarize yourself with the concept we'll use here. This tutorial also assumes familiarity with the `tf.keras` API. Specifically it is based on the [introduction in the Tensorflow documentation](https://www.tensorflow.org/tutorials/keras/basic_classification), so if you are not yet familiar with Keras, you may find it helpful to read those tutorials first as well.

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import nengo
import nengo_dl

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


4423680/4422102 [==============================] - 1s 0us/step


In [3]:
model = keras.Sequential([
            keras.layers.Flatten(input_shape=(28, 28)),
            keras.layers.Dense(128, activation=tf.nn.relu),
            keras.layers.Dense(10, activation=tf.nn.softmax)
        ])

model.compile(optimizer=tf.train.AdamOptimizer(), 
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [4]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [5]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 8s 140us/step - loss: 0.5025 - acc: 0.8240
Epoch 2/5
60000/60000 [==============================] - 4s 75us/step - loss: 0.3739 - acc: 0.8641
Epoch 3/5
60000/60000 [==============================] - 5s 83us/step - loss: 0.3352 - acc: 0.8774
Epoch 4/5
60000/60000 [==============================] - 5s 84us/step - loss: 0.3106 - acc: 0.8863
Epoch 5/5
60000/60000 [==============================] - 5s 84us/step - loss: 0.2923 - acc: 0.8912


In [8]:
model.save_weights('fully_connected_weights.h5')

In [49]:
model_weights = 'fully_connected_weights.h5'
image_shape = (28, 28)

class FullyConnectedNode(object):
    def pre_build(self, *args):
        self.model = keras.Sequential([
            keras.layers.Flatten(input_shape=image_shape),
            keras.layers.Dense(128, activation=tf.nn.relu),
            keras.layers.Dense(10, activation=tf.nn.softmax)
        ])

    def post_build(self, sess, rng):
        # load checkpoint file into model
        self.model.compile(optimizer=tf.train.AdamOptimizer(), 
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        self.model.load_weights(model_weights)

    def __call__(self, t, x):
        # this is the function that will be executed each timestep 
        # while the network is running
                
        # convert our input vector to the shape/dtype of the input image
        images = tf.reshape(x, (-1,) + image_shape)
        images = tf.expand_dims(image, 0)

        probabilities = self.model.predict(images)
        return probabilities

In [50]:
train_shape = train_images.shape

with nengo.Network() as net:
    # create a normal input node to feed in our test image
    input_node = nengo.Node([0] * np.prod(train_shape[1:]))

    # create our TensorNode containing the FullyConnectedNode() we defined
    # above.  we also need to specify size_in (the dimensionality of
    # our input vectors, the flattened images) and size_out (the number
    # of classification classes output by the inception network)
    fc_node = nengo_dl.TensorNode(
        FullyConnectedNode(),
        size_in=np.prod(train_shape[1:]),
        size_out=10)

    # connect up our input to our fully-connected etwork node
    nengo.Connection(input_node, fc_node, synapse=None)

    # add some probes to collect data
    input_p = nengo.Probe(input_node)
    fc_p = nengo.Probe(fc_node)

In [51]:
minibatch_size = 10
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


/home/ildefonso/Documents/repositories/open-source-dev/nengo-dl-NickleDave/nengo_dl/simulator.py:1612: RuntimeWarning: Simulator with model=Model: <Network (unlabeled) at 0x7fa57ca14c50>, dt=0.001000 was deallocated while open. Simulators should be closed manually to ensure resources are properly freed.
  "are properly freed." % self.model, RuntimeWarning)
/home/ildefonso/Documents/repositories/open-source-dev/nengo-dl-NickleDave/nengo_dl/simulator.py:1612: RuntimeWarning: Simulator with model=Model: <Network (unlabeled) at 0x7fa5799c1160>, dt=0.001000 was deallocated while open. Simulators should be closed manually to ensure resources are properly freed.
  "are properly freed." % self.model, RuntimeWarning)
/home/ildefonso/Documents/repositories/open-source-dev/nengo-dl-NickleDave/nengo_dl/simulator.py:1612: RuntimeWarning: Simulator with model=Model: <Network (unlabeled) at 0x7fa5784824a8>, dt=0.001000 was deallocated while open. Simulators should be closed manually to ensure resourc

NameError: name 'image' is not defined

In [31]:
# run the network for one timestep
with nengo_dl.Simulator(net) as sim:
    sim.step()

Build finished in 0:00:00                                                      


/home/ildefonso/Documents/repositories/open-source-dev/nengo-dl-NickleDave/nengo_dl/simulator.py:134: UserWarning: No GPU support detected. It is recommended that you install tensorflow-gpu (`pip install tensorflow-gpu`).
  "No GPU support detected. It is recommended that you "


BuildError: Building <Connection from <TensorNode (unlabeled) at 0x7fa57998ff28> to <Probe of 'output' of <TensorNode (unlabeled) at 0x7fa57998ff28>>>: the 'pre' object <TensorNode (unlabeled) at 0x7fa57998ff28> is not in the model, or has a size of zero.